In [ ]:
from src.models import BoundingBox, Vertex
from src.services import DataConverterService, ImageDisplayService, LineDetectionService

from config import config

"""
    getting a single datapoint from the dataset 
"""

# use this when you want to load an image from the dataset whose path come frome the config files.
# converter_service = DataConverterService() 
# datapoint = converter_service.load_single_datapoint(str(0))

In [ ]:
from src.services.predict_symbols_service import PredictSymbolsService
from src.models.symbol import Symbol
from src.utils.convert_points_to_bounding_box import convert_points_to_bounding_box


"""
    predict the symbol bounding boxes using the prediction service.
"""

image_path_to_test = "D:/Veilex/_test/crop-data/_images/19.jpg"

predicted_symbol_results = []

predict_service = PredictSymbolsService(
    image_path=image_path_to_test,
    model_path="./yolo-model-pid.pt"
)

prediction_results = predict_service.predict_bounding_boxes(shifting=False)

for index, pr in enumerate(prediction_results):
    symbol = Symbol(
        name=f"s-{str(index)}",
        label=pr[1],
        pointSrc=Vertex(x=pr[0][0], y=pr[0][1]),
        pointDest=Vertex(x=pr[0][2], y=pr[0][3])
    )

    predicted_symbol_results.append(symbol)

In [ ]:


"""
    predict the word bounding boxes using the azure document inteligence service.
"""

from src.services.predict_word_service import PredictWordService

word_bboxes = []
predict_word_service = PredictWordService(
    image_path=image_path_to_test
)

result = predict_word_service.predicit_bounding_boxes()

for index, item in enumerate(result): 
    word_bboxes.append(
        BoundingBox(
            name=f"w-{index}",
            pointSrc=Vertex(x=item[0],y=item[1]),
            pointDest=Vertex(x=item[2],y=item[3])
        )
    )

In [ ]:
display_service = ImageDisplayService(
    image_path_to_test,
    [*predicted_symbol_results, *word_bboxes]
)
display_service.display_image_with_bbox(color='red', dpi=200, show_text=False)

In [ ]:
from src.utils.calculate_distance_between_rectangles import calculate_distance_between_rectangles

"""
    remove redundent bounding boxes.
"""

index_to_delete = []

for index_1, s1 in enumerate(predicted_symbol_results):
    for index_2, s2 in enumerate(predicted_symbol_results[index_1 + 1:]):
        if(
            calculate_distance_between_rectangles(
                [*s1.pointSrc.get_dimensions(), *s1.pointDest.get_dimensions()], 
                [*s2.pointSrc.get_dimensions(), *s2.pointDest.get_dimensions()]                
            ) < 5
        ):
            index_to_delete.append(index_1)


predicted_symbol_results = [value for i, value in enumerate(predicted_symbol_results) if i not in list(set(index_to_delete))]

predicted_symbol_results

In [ ]:
# line detection and their extension from the datapoint
line_detection_service = LineDetectionService(
    image_path=image_path_to_test,
    bounding_boxes=[*predicted_symbol_results, *word_bboxes],
)

def get_lines(merge=True):
    if not merge:
        return [convert_points_to_bounding_box(l) for l in line_detection_service.extend_lines(
            line_detection_service.detect_line_segments(enable_thining=True)      
        )]
    else:
        return [convert_points_to_bounding_box(l) for l in 
            line_detection_service.merge_lines(
                line_segments = line_detection_service.extend_lines(
                    line_detection_service.detect_line_segments(enable_thining=True)       
                )
            )
        ]

line_segments = get_lines(True)

for index, l in enumerate(line_segments):
    l.name = f"l-{str(index)}"

line_segments

In [ ]:
# filter the smaller lines, make them bigger than ususal. Implement them in the line service afterwards.

# from src.utils.calculate_distance_between_points import calculate_distance_between_points

# short_line_semgents = list(filter(lambda l: l.name in ['l-0', 'l-10'], line_segments))

# print(short_line_semgents)
# [calculate_distance_between_points(s.pointSrc.get_dimensions(), s.pointDest.get_dimensions()) for s in short_line_semgents]

In [ ]:
# *predicted_symbol_results

line_display_service = ImageDisplayService(
    image_path_to_test,
    [*line_segments, *predicted_symbol_results, *word_bboxes]
)

line_display_service.display_image_with_bbox(color='red', dpi=300, show_text=False)

In [ ]:
from src.services.graph_construction_service import GraphConstructionService

graph_service = GraphConstructionService(predicted_symbol_results, line_segments)
graph_service.initialize_graph()
graph_service.define_graph_edges()

In [ ]:
service = ImageDisplayService()
service.display_graph(graph_service.graph)

In [ ]:
graph_service.reduce_line_cycles()
# graph_service.set_largest_graph_connected_nodes()

In [ ]:
for i in range(100):
    graph_service.remove_zero_or_single_connection_line_nodes()

In [ ]:
# since i have worked on merging the lines, I might not use this.
# graph_service.prune_multiple_path_nodes(graph_service.find_valid_paths())

In [ ]:
service.display_graph(graph_service.graph)

In [ ]:
# this utility might not be needed anymore.

import networkx as nx
# this can be used for utility purposes.

def subgraph_between_nodes(G, start_node, end_node):
    # Find a simple path between the two nodes
    paths = list(nx.all_simple_paths(G, source=start_node, target=end_node))
    
    # Create a subgraph from the first valid path
    if paths:
        path = paths[0]
        subgraph = G.subgraph(path).copy()
        
        # Remove any edges that create duplicate connections
        for node in path:
            neighbors = list(subgraph.neighbors(node))
            # Only keep the neighbor that is part of the path
            for neighbor in neighbors:
                if neighbor not in path:
                    subgraph.remove_edge(node, neighbor)
                    
        return subgraph
    else:
        return None


service.display_graph(
    subgraph_between_nodes(graph_service.graph, "l-16", "l-6")
)